In [2]:
import pandas as pd


df = pd.read_csv('dataset\dataset.csv')


print(df.head())


          userName  score                   at  \
0  Pengguna Google      5  2024-09-08 03:31:50   
1  Pengguna Google      1  2024-09-08 03:29:56   
2  Pengguna Google      5  2024-09-08 03:26:38   
3  Pengguna Google      5  2024-09-08 03:25:14   
4  Pengguna Google      1  2024-09-08 03:24:01   

                                             content  
0                                      makasih toped  
1  Aplikasi php sudah banyak dikasih promo &sudah...  
2                                        mantab... 👍  
3                                          Good good  
4  Sangat buruk sebagai pengguna lama akun affali...  


In [3]:

def label_sentiment(score):
    if score >= 4:
        return 'Positif'
    elif score == 3:
        return 'Netral'
    else:
        return 'Negatif'

df['sentiment'] = df['score'].apply(label_sentiment)


print(df[['content', 'score', 'sentiment']].head())


                                             content  score sentiment
0                                      makasih toped      5   Positif
1  Aplikasi php sudah banyak dikasih promo &sudah...      1   Negatif
2                                        mantab... 👍      5   Positif
3                                          Good good      5   Positif
4  Sangat buruk sebagai pengguna lama akun affali...      1   Negatif


In [4]:
import re
import pandas as pd


def preprocess_text(text):
    if isinstance(text, str): 
        text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)  
        text = text.lower().strip()  
        return text
    else:
        return ''  


df['content'] = df['content'].fillna('')


df['cleaned_content'] = df['content'].apply(preprocess_text)


print(df[['content', 'cleaned_content']].head())


                                             content  \
0                                      makasih toped   
1  Aplikasi php sudah banyak dikasih promo &sudah...   
2                                        mantab... 👍   
3                                          Good good   
4  Sangat buruk sebagai pengguna lama akun affali...   

                                     cleaned_content  
0                                      makasih toped  
1  aplikasi php sudah banyak dikasih promo sudah ...  
2                                             mantab  
3                                          good good  
4  sangat buruk sebagai pengguna lama akun affali...  


In [5]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p2")


train_encodings = tokenizer(df['cleaned_content'].tolist(), truncation=True, padding=True, max_length=128)


print(train_encodings['input_ids'][:2])


d:\sentiment\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[2, 6679, 2246, 133, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1339, 12648, 259, 271, 8852, 4525, 259, 6039, 469, 316, 1192, 11750, 16850, 737, 3854, 12510, 737, 22180, 14813, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [6]:
from sklearn.model_selection import train_test_split


train_texts, test_texts, train_labels, test_labels = train_test_split(df['cleaned_content'], df['sentiment'], test_size=0.2, stratify=df['sentiment'])


train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)


In [7]:
import torch


label_dict = {'Negatif': 0, 'Netral': 1, 'Positif': 2}
train_labels = train_labels.map(label_dict)
test_labels = test_labels.map(label_dict)


class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
test_dataset = SentimentDataset(test_encodings, test_labels.tolist())


# IndoBERT

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments


model = BertForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p2", num_labels=3)


training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=3, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=64, 
    warmup_steps=500, 
    weight_decay=0.01, 
    logging_dir='./logs', 
    logging_steps=10, 
    evaluation_strategy="epoch",  
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


trainer.train()


# TinyBert

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")


train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)


train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
test_dataset = SentimentDataset(test_encodings, test_labels.tolist())


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 2/75000 [01:24<880:59:48, 42.29s/it]
                                                   
  0%|          | 12/75000 [00:00<1:08:19, 18.29it/s]

{'loss': 1.0988, 'grad_norm': 1.3116143941879272, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


                                                    
  0%|          | 24/75000 [00:01<53:30, 23.35it/s]  

{'loss': 1.0944, 'grad_norm': 1.7751038074493408, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


                                                  
  0%|          | 33/75000 [00:01<51:48, 24.11it/s]  

...

                                                       
100%|██████████| 75000/75000 [1:12:51<00:00, 13.50it/s]

{'loss': 0.2709, 'grad_norm': 1.4303580522537231, 'learning_rate': 0.0, 'epoch': 3.0}




















































































































100%|██████████| 75000/75000 [1:13:07<00:00, 13.50it/s]






















































































































































































































































































































































































































                                                       
                                                   

100%|██████████| 75000/75000 [1:13:58<00:00, 13.50it/s]

                                                       
100%|██████████| 75000/75000 [1:13:58<00:00, 16.90it/s]

{'eval_loss': 0.32544657588005066, 'eval_runtime': 66.3771, 'eval_samples_per_second': 1506.543, 'eval_steps_per_second': 23.547, 'epoch': 3.0}
{'train_runtime': 4438.7153, 'train_samples_per_second': 270.348, 'train_steps_per_second': 16.897, 'train_loss': 0.3269071028693517, 'epoch': 3.0}


TrainOutput(global_step=75000, training_loss=0.3269071028693517, metrics={'train_runtime': 4438.7153, 'train_samples_per_second': 270.348, 'train_steps_per_second': 16.897, 'total_flos': 4301987328000000.0, 'train_loss': 0.3269071028693517, 'epoch': 3.0})

In [ ]:
import torch
print(torch.cuda.is_available())  # Harus True jika GPU tersedia
print(torch.cuda.get_device_name(0))  # Menampilkan nama GPU, misalnya 'NVIDIA GeForce RTX 2060'


In [14]:

eval_results = trainer.evaluate(test_dataset)


print(eval_results)


100%|██████████| 1563/1563 [00:57<00:00, 27.36it/s]

{'eval_loss': 0.32544657588005066, 'eval_runtime': 57.1679, 'eval_samples_per_second': 1749.233, 'eval_steps_per_second': 27.341, 'epoch': 3.0}


In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)


accuracy = accuracy_score(test_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, preds, average='weighted')


print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


100%|██████████| 1563/1563 [01:00<00:00, 26.03it/s]


Accuracy: 0.90118
Precision: 0.8832453423991299
Recall: 0.90118
F1-score: 0.8861208356918333


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


cm = confusion_matrix(test_labels, preds)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [11]:

model.save_pretrained("./saved_model")


tokenizer.save_pretrained("./saved_model")


('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.txt',
 './saved_model\\added_tokens.json',
 './saved_model\\tokenizer.json')

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


loaded_model = AutoModelForSequenceClassification.from_pretrained("./saved_model")


loaded_tokenizer = AutoTokenizer.from_pretrained("./saved_model")


In [13]:

inputs = loaded_tokenizer("Contoh teks untuk prediksi", return_tensors="pt")


outputs = loaded_model(**inputs)
predictions = outputs.logits.argmax(-1)
print(predictions)


tensor([2])


In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


model_path = "./saved_model"  
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_path)


loaded_tokenizer = AutoTokenizer.from_pretrained(model_path)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-1

In [18]:

text = "Layanan pelanggan sangat buruk, saya tidak akan membeli lagi."


inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)


inputs = {key: val.to(device) for key, val in inputs.items()}


outputs = loaded_model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)


label_dict = {0: 'Negatif', 1: 'Netral', 2: 'Positif'}
predicted_label = label_dict[predictions.item()]

print(f"Teks: {text}")
print(f"Prediksi Sentimen: {predicted_label}")


Teks: Layanan pelanggan sangat buruk, saya tidak akan membeli lagi.
Prediksi Sentimen: Negatif


In [19]:

texts = [
    "Layanan pelanggan sangat buruk, saya tidak akan membeli lagi.",
    "Produk ini sangat bagus, saya sangat puas dengan pembeliannya.",
    "Pengiriman cukup lama tapi barangnya sesuai deskripsi."
]


inputs = loaded_tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=128)


inputs = {key: val.to(device) for key, val in inputs.items()}


outputs = loaded_model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)


predicted_labels = [label_dict[pred.item()] for pred in predictions]

for text, label in zip(texts, predicted_labels):
    print(f"Teks: {text}")
    print(f"Prediksi Sentimen: {label}")


Teks: Layanan pelanggan sangat buruk, saya tidak akan membeli lagi.
Prediksi Sentimen: Negatif
Teks: Produk ini sangat bagus, saya sangat puas dengan pembeliannya.
Prediksi Sentimen: Positif
Teks: Pengiriman cukup lama tapi barangnya sesuai deskripsi.
Prediksi Sentimen: Positif
